In [1]:
import gym
import torch
%load_ext autoreload
%autoreload 2

In [16]:
from rl.models.action_value import ExpectedSarsa
from rl.strategies.function_approximation import EpsilonGreedy

In [183]:
env = gym.make('LunarLander-v2')
strat = EpsilonGreedy({
	"epsilon": 0.1,
	"lr": 1e-2,
	"env": env,
	"seed": 72047,
	"weight_decay": 1e-5,
	"action_value": {
		"layers": [6]
	}
})
agent = ExpectedSarsa(env, {
	"policy": {
		"layers": [4]
	},
	"step_size": 1e-2,
	"batch_size": 2048,
	"gamma": 0.99
}, strat)

observation, info = env.reset(return_info=True)
observation = torch.from_numpy(observation).float()

In [184]:
from tqdm import tqdm
episodes = 3000
render_every = 100
rolling_avg_window = 100
agent.reset()
ep_rewards = []
loop = tqdm(range(episodes))
for epoch in loop:
	observation, info = env.reset(return_info=True)
	observation = torch.from_numpy(observation).float()
	ep_reward = 0
	while True:
		action = agent.get_action(observation)
		n_observation, reward, done, _= env.step(action)
		n_observation = torch.from_numpy(n_observation).float()
		ep_reward += reward
		# reward = reward ** 5
		agent.update(observation, action, reward, n_observation, done)
		observation = n_observation
		if epoch % render_every == 0:
			env.render()
		if done:
			break
	ep_rewards.append(ep_reward)
	rolling_avg_rewards = ep_rewards[-rolling_avg_window:]
	avg = sum(rolling_avg_rewards)/len(rolling_avg_rewards)
	loop.set_description(f"{rolling_avg_window}eps avg reward: {avg}")

100eps avg reward: -79.42642933485601:  61%|██████    | 1824/3000 [18:03<11:38,  1.68it/s]   


KeyboardInterrupt: 

In [163]:
env.close()

In [131]:
random.random()

0.9200848237332326